In [ ]:
## potrebbe essere interessante parsarlo in un dataframe per avere i risultati come dati tabellari
import json
import os
import sys
import re
import pandas as pd
sys.path.append('../../')

# with open(path, "r") as read_file:
#     data = json.load(read_file)
# 
# print(data)

In [ ]:
def parse_json():   
    path = "../great_expectations/uncommitted/validations/Main"
    #egex1 = r"/(\d{8})-"
    #craping_date=[]
    expectation_type=[]
    dataframe_type=[]
    expectation_type=[]
    element_count=[]
    missing_count=[]
    missing_percent=[]
    partial_unexpected_counts=[]
    partial_unexpected_index_list=[]
    partial_unexpected_list=[]
    unexpected_count=[]
    unexpected_percent=[]
    unexpected_percent_nonmissing=[]
    unexpected_percent_total=[]
    column=[]
    column_list=[]
    success=[]
    timestamp_run=[]
    
    
    for root, dirs, files in os.walk(path):
        for file in files:
            path_data = os.path.join(root, file)
            #escraping_date=re.search(regex1, path_data).group(1)
            
            with open(path_data, "r") as read_file:
                data = json.load(read_file)
                str_dataframe_type = data['meta']['active_batch_definition']['data_asset_name']
                str_timestamp_run = data['meta']['batch_markers']['ge_load_time']
                        
                    
                for e in data['results']:
                    for i in range(4):
                        dataframe_type.append(str_dataframe_type)
                        timestamp_run.append(str_timestamp_run)
                    
                    
                    

                    
                    try:
                        expectation_type.append(e['expectation_config']['expectation_type'])
                    except:
                        expectation_type.append(None)

                    try:
                        column.append(e['expectation_config']['kwargs']['column'])
                    except:
                        column.append(e['expectation_config']['kwargs']['column_list'][0])    
                    try:
                        column_list.append(e['expectation_config']['kwargs']['column_list'])
                    except:
                        column_list.append(None)          
                    try:
                        element_count.append(e['result']['element_count'])
                    except:
                        element_count.append(None)
                    try:
                        missing_count.append(e['result']['missing_count'])
                    except:
                        missing_count.append(None)
                    try:
                        missing_percent.append(e['result']['missing_percent'])
                    except:
                        missing_percent.append(None)
                    try:
                        partial_unexpected_counts.append(e['result']['partial_unexpected_counts'])
                    except:
                        partial_unexpected_counts.append(None)
                    try:
                        partial_unexpected_index_list.append(e['result']['partial_unexpected_index_list'])
                    except:
                        partial_unexpected_index_list.append(None)
                    try:
                        partial_unexpected_list.append(e['result']['partial_unexpected_list'])
                    except:
                        partial_unexpected_list.append(None)
                    try:
                        unexpected_count.append(e['result']['unexpected_count'])
                    except:
                        unexpected_count.append(None)
                    try:
                        unexpected_percent.append(e['result']['unexpected_percent'])
                    except:
                        unexpected_percent.append(None)
                    try:
                        unexpected_percent_nonmissing.append(e['result']['unexpected_percent_nonmissing'])
                    except:
                        unexpected_percent_nonmissing.append(None)
                    try:
                        unexpected_percent_total.append(e['result']['unexpected_percent_total'])
                    except:
                        unexpected_percent_total.append(None)
                    try:
                        expectation_type.append(e['expectation_config']['expectation_type'])
                    except:
                        expectation_type.append(None)
                    try:
                        column.append(e['expectation_config']['kwargs']['column'])
                    except:
                        column.append(e['expectation_config']['kwargs']['column_list'][0])    
                    try:
                        column_list.append(e['expectation_config']['kwargs']['column_list'])
                    except:
                        column_list.append(None)          
                    try:
                        element_count.append(e['result']['element_count'])
                    except:
                        element_count.append(None)
                    try:
                        missing_count.append(e['result']['missing_count'])
                    except:
                        missing_count.append(None)
                    try:
                        missing_percent.append(e['result']['missing_percent'])
                    except:
                        missing_percent.append(None)
                    try:
                        partial_unexpected_counts.append(e['result']['partial_unexpected_counts'])
                    except:
                        partial_unexpected_counts.append(None)
                    try:
                        partial_unexpected_index_list.append(e['result']['partial_unexpected_index_list'])
                    except:
                        partial_unexpected_index_list.append(None)
                    try:
                        partial_unexpected_list.append(e['result']['partial_unexpected_list'])
                    except:
                        partial_unexpected_list.append(None)
                    try:
                        unexpected_count.append(e['result']['unexpected_count'])
                    except:
                        unexpected_count.append(None)
                    try:
                        unexpected_percent.append(e['result']['unexpected_percent'])
                    except:
                        unexpected_percent.append(None)
                    try:
                        unexpected_percent_nonmissing.append(e['result']['unexpected_percent_nonmissing'])
                    except:
                        unexpected_percent_nonmissing.append(None)
                    try:
                        unexpected_percent_total.append(e['result']['unexpected_percent_total'])
                    except:
                        unexpected_percent_total.append(None)
                    try:
                        expectation_type.append(e['expectation_config']['expectation_type'])
                    except:
                        expectation_type.append(None)
                    try:
                        column.append(e['expectation_config']['kwargs']['column'])
                    except:
                        column.append(e['expectation_config']['kwargs']['column_list'][0])    
                    try:
                        column_list.append(e['expectation_config']['kwargs']['column_list'])
                    except:
                        column_list.append(None)          
                    try:
                        element_count.append(e['result']['element_count'])
                    except:
                        element_count.append(None)
                    try:
                        missing_count.append(e['result']['missing_count'])
                    except:
                        missing_count.append(None)
                    try:
                        missing_percent.append(e['result']['missing_percent'])
                    except:
                        missing_percent.append(None)
                    try:
                        partial_unexpected_counts.append(e['result']['partial_unexpected_counts'])
                    except:
                        partial_unexpected_counts.append(None)
                    try:
                        partial_unexpected_index_list.append(e['result']['partial_unexpected_index_list'])
                    except:
                        partial_unexpected_index_list.append(None)
                    try:
                        partial_unexpected_list.append(e['result']['partial_unexpected_list'])
                    except:
                        partial_unexpected_list.append(None)
                    try:
                        unexpected_count.append(e['result']['unexpected_count'])
                    except:
                        unexpected_count.append(None)
                    try:
                        unexpected_percent.append(e['result']['unexpected_percent'])
                    except:
                        unexpected_percent.append(None)
                    try:
                        unexpected_percent_nonmissing.append(e['result']['unexpected_percent_nonmissing'])
                    except:
                        unexpected_percent_nonmissing.append(None)
                    try:
                        unexpected_percent_total.append(e['result']['unexpected_percent_total'])
                    except:
                        unexpected_percent_total.append(None)
                    try:
                        expectation_type.append(e['expectation_config']['expectation_type'])
                    except:
                        expectation_type.append(None)
                    try:
                        column.append(e['expectation_config']['kwargs']['column'])
                    except:
                        column.append(e['expectation_config']['kwargs']['column_list'][0])    
                    try:
                        column_list.append(e['expectation_config']['kwargs']['column_list'])
                    except:
                        column_list.append(None)          
                    try:
                        element_count.append(e['result']['element_count'])
                    except:
                        element_count.append(None)
                    try:
                        missing_count.append(e['result']['missing_count'])
                    except:
                        missing_count.append(None)
                    try:
                        missing_percent.append(e['result']['missing_percent'])
                    except:
                        missing_percent.append(None)
                    try:
                        partial_unexpected_counts.append(e['result']['partial_unexpected_counts'])
                    except:
                        partial_unexpected_counts.append(None)
                    try:
                        partial_unexpected_index_list.append(e['result']['partial_unexpected_index_list'])
                    except:
                        partial_unexpected_index_list.append(None)
                    try:
                        partial_unexpected_list.append(e['result']['partial_unexpected_list'])
                    except:
                        partial_unexpected_list.append(None)
                    try:
                        unexpected_count.append(e['result']['unexpected_count'])
                    except:
                        unexpected_count.append(None)
                    try:
                        unexpected_percent.append(e['result']['unexpected_percent'])
                    except:
                        unexpected_percent.append(None)
                    try:
                        unexpected_percent_nonmissing.append(e['result']['unexpected_percent_nonmissing'])
                    except:
                        unexpected_percent_nonmissing.append(None)
                    try:
                        unexpected_percent_total.append(e['result']['unexpected_percent_total'])
                    except:
                        unexpected_percent_total.append(None)
                        
                    
    
    df_expectations = pd.DataFrame() 

    df_expectations['column'] = column
    df_expectations['column_list'] = column_list
    df_expectations['expectation_type']=expectation_type
    df_expectations['element_count'] = element_count
    df_expectations['missing_count'] = missing_count
    df_expectations['missing_percent'] = missing_percent
    df_expectations['partial_unexpected_counts'] = partial_unexpected_counts
    df_expectations['partial_unexpected_index_list'] = partial_unexpected_index_list
    df_expectations['partial_unexpected_list'] = partial_unexpected_list
    df_expectations['unexpected_count'] = unexpected_count
    df_expectations['unexpected_percent'] = unexpected_percent
    df_expectations['unexpected_percent_nonmissing'] = unexpected_percent_nonmissing
    df_expectations['unexpected_percent_total'] = unexpected_percent_total
    df_expectations['dataframe_type']=dataframe_type
    df_expectations['timestamp_run_quality']=timestamp_run
    #f_expectations['date_run_quality']=timestamp_run[0:8]
    #f_expectations['scraping_date']=scraping_date

    #f_expectations['scraping_date'] = pd.to_datetime(df_expectations['scraping_date'], format='%Y%m%d').dt.strftime('%Y-%m-%d')
    df_expectations = df_expectations[ ['dataframe_type'] + ['timestamp_run_quality']+ [ col for col in df_expectations.columns if (col != 'dataframe_type')|(col != 'timestamp_run_quality')] ]
    return df_expectations.iloc[:,0:-4]


In [ ]:
df = parse_json()